## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [41]:
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000252A3803910>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000252A38033A0>, model_name='gemma2-9b-it')

In [42]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, My name is Vishal and I am a Chief AI Engineer")])

AIMessage(content="Hello Vishal, it's a pleasure to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What are you currently working on that excites you the most?\n\nI'm eager to learn more about your work and the challenges and opportunities you face in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 23, 'total_tokens': 86, 'completion_time': 0.114545455, 'prompt_time': 0.00133619, 'queue_time': 0.257187999, 'total_time': 0.115881645}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a89224ba-a535-409b-ab5c-d3b07c57aa5e-0', usage_metadata={'input_tokens': 23, 'output_tokens': 63, 'total_tokens': 86})

In [43]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, My name is Vishal and I am a Chief AI Engineer"),
        AIMessage(content="Hi Vishal, it's nice to meet you!  \n\nBeing a Chief AI Engineer is a fascinating role. What are you working on these days? Are there any particular challenges or exciting projects you're involved in?  \n\nI'm always eager to learn more about the applications of AI."),
        HumanMessage(content="Hey Whats my name and what do I do?")
    ]
)

AIMessage(content="You are Vishal, and you are a Chief AI Engineer! 🤖  \n\nDo you need me to remind you of anything else about your role? 😊  Perhaps you'd like to discuss a project you're working on? 💡  I'm here to help!\n", response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 105, 'total_tokens': 166, 'completion_time': 0.110909091, 'prompt_time': 0.002814479, 'queue_time': 0.25498937, 'total_time': 0.11372357}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8471dc7f-9e68-46f8-8061-28ad015d596c-0', usage_metadata={'input_tokens': 105, 'output_tokens': 61, 'total_tokens': 166})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [44]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [45]:
config={"configurable":{"session_id":"chat1"}}

In [46]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Vishal and I am a Chief AI Engineer")],
    config=config
)

In [47]:
response.content

"Hello Vishal, it's a pleasure to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on?  \n\nI'm eager to learn more about your work in the field of AI.\n"

In [48]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Vishal.  \n\nI remember that from our earlier conversation! 😊  Is there anything else I can help you with today?\n', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 92, 'total_tokens': 124, 'completion_time': 0.058181818, 'prompt_time': 0.002598059, 'queue_time': 0.25718823, 'total_time': 0.060779877}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5b52b8ad-434a-4ff4-9411-8960852d868d-0', usage_metadata={'input_tokens': 92, 'output_tokens': 32, 'total_tokens': 124})

In [ ]:
## Change the config ---> session_id

config1={"configurable":{"session_id":"chat2"}}

response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)

response.content

'As an AI, I have no memory of past conversations and do not know your name. Could you please tell me?\n'

In [50]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! 👋 \n\nHow can I help you today?\n"

In [51]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, remember? 😊  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [52]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [53]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Vishal")]})

AIMessage(content="Hello Vishal, it's nice to meet you! \n\nI'm ready to answer your questions to the best of my ability.  \n\nWhat can I help you with today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 31, 'total_tokens': 76, 'completion_time': 0.081818182, 'prompt_time': 0.001479849, 'queue_time': 0.254046611, 'total_time': 0.083298031}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-016110cb-3b4e-4836-903f-c3cfd103a953-0', usage_metadata={'input_tokens': 31, 'output_tokens': 45, 'total_tokens': 76})

In [54]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [55]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Vishal")],
    config=config
)

response

AIMessage(content="Hi Vishal, it's nice to meet you!\n\nI'm happy to answer any questions you have to the best of my ability.  What can I help you with today? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 31, 'total_tokens': 75, 'completion_time': 0.08, 'prompt_time': 0.001485599, 'queue_time': 0.26580043, 'total_time': 0.081485599}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b5a5bd7c-5788-47d1-a734-f1c85c77a202-0', usage_metadata={'input_tokens': 31, 'output_tokens': 44, 'total_tokens': 75})

In [56]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Vishal.  I remember that you told me! 😊  \n\nIs there anything else I can help you with?\n'

In [61]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [62]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Vishal")],"language":"Hindi"})
response.content

'नमस्ते विशाल, \n\nमुझे खुशी है आपसे मिलने की। मैं आपकी मदद करने के लिए तैयार हूँ।  \n\nआप से क्या पूछना है?  \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [63]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [64]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Vishal")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते विशाल! \n\nमैं आपकी मदद करने के लिए यहाँ हूँ। आप किसी भी प्रश्न पूछ सकते हैं और मैं उसका उत्तर देने की पूरी कोशिश करूँगा। 😊 \n'

In [65]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [66]:
response.content

'आपका नाम विशाल है।  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [67]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 45,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial =False,
    start_on = "human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

d:\GenAI stuff\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\GenAI stuff\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microso

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [69]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")| trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="what ice cream do i like")],
        "language":"English"
    }
)

response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences.  \n\nWhat's your favorite flavor?  🍨\n"

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "What is 2 + 2?"  😊\n'

In [73]:
## Lets wrap this iin the message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable":{"session_id":"chat5"}}

In [74]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English"
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or personal information about you. I need you to tell me your name! 😊  What's it?\n"

In [75]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help!  😊 \n\n"